In [1]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from constant import FeatureType
from loader import Power, Weather
from model import Model

import argparse

### Experiment

In [3]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Path ====== #
args.root = root
args.experiment_name = "LSTM-003"
args.automl_name = 'automl-003'

# ====== Model ====== #
args.frame_in = 72
args.frame_out = 24
args.epochs = 200
args.learning_rate = 0.001
args.patience = 30

# ====== Data ====== #
args.years = [2017, 2018, 2019]
args.region = "Jindo"
args.station = 192
args.ratio = [0.6, 0.2, 0.2]

# ====== Features ====== #
features = [FeatureType.SUNSHINE,
            FeatureType.GROUND_TEMPERATURE,
            FeatureType.HUMIDITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.PRECIPITATION,
            FeatureType.STEAM_PRESSURE,
            FeatureType.DEW_POINT_TEMPERATURE,
            FeatureType.ATMOSPHERIC_PRESSURE]

power = Power(args)
weather = Weather(args, features)

power_data = power.get_data()
model_manager = Model(power_data['scaler'])
setattr(args, 'test_start', power.test_start)
setattr(args, 'test_end', power.test_end)

y_pred_list = []
results = []
for i in range(len(features)):
    weather_data = weather.get_data(i + 1)
    setattr(args, 'feature_len', i + 1)
    setattr(args, 'name', 'feature-%02d' % (i + 1))

    train = [weather_data['train'], power_data['train']]
    val = [weather_data['val'], power_data['val']]
    test = [weather_data['test'], power_data['test']]

    dataset = {'train': train, 'val': val, 'test': test}
    args = model_manager.read_setting(args)
    model = model_manager.train(args, dataset)
    y_pred = model.predict(dataset['test'][0])
    y_pred_list.append(y_pred)
    result = model_manager.evaluate(model, dataset['test'][0], dataset['test'][1])
    results.append(result)

setattr(args, 'features', features)
model_manager.save_result(dataset['test'][1], y_pred_list, args)

train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
(15768,)
(5256,)
(5256,)
missing dates: []
missing dates: ['2018-01-13 09:00', '2018-01-13 10:00', '2018-01-13 11:00', '2018-01-13 12:00', '2018-01-13 13:00']
missing dates: []
Epoch 1/200
 7/11 [==================>...........] - ETA: 0s - loss: 0.0116
Epoch 00001: val_loss improved from inf to 0.00138, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-003/feature-01/model-001-0.001384.h5
11/11 [=============================

 8/11 [====================>.........] - ETA: 0s - loss: 5.2460e-04
Epoch 00030: val_loss did not improve from 0.00034
11/11 [==============================] - 0s 10ms/step - loss: 4.9924e-04 - val_loss: 6.9828e-04
Epoch 31/200
 8/11 [====================>.........] - ETA: 0s - loss: 5.5835e-04
Epoch 00031: val_loss did not improve from 0.00034
11/11 [==============================] - 0s 10ms/step - loss: 5.2485e-04 - val_loss: 3.8279e-04
Epoch 32/200
 8/11 [====================>.........] - ETA: 0s - loss: 5.4648e-04
Epoch 00032: val_loss improved from 0.00034 to 0.00031, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-003/feature-01/model-032-0.000315.h5
11/11 [==============================] - 0s 11ms/step - loss: 5.0852e-04 - val_loss: 3.1481e-04
Epoch 33/200
 8/11 [====================>.........] - ETA: 0s - loss: 5.1746e-04
Epoch 00033: val_loss did not improve from 0.00031
11/11 [==============================] - 0s 10ms/step - loss: 5.1184e-04 - val_loss: 

 8/11 [====================>.........] - ETA: 0s - loss: 4.6412e-04
Epoch 00065: val_loss did not improve from 0.00029
11/11 [==============================] - 0s 10ms/step - loss: 4.5262e-04 - val_loss: 3.4468e-04
Epoch 66/200
 8/11 [====================>.........] - ETA: 0s - loss: 4.6507e-04
Epoch 00066: val_loss did not improve from 0.00029
11/11 [==============================] - 0s 10ms/step - loss: 4.5352e-04 - val_loss: 4.9777e-04
Epoch 67/200
 7/11 [==================>...........] - ETA: 0s - loss: 4.6336e-04
Epoch 00067: val_loss did not improve from 0.00029
11/11 [==============================] - 0s 10ms/step - loss: 4.3399e-04 - val_loss: 5.4041e-04
Epoch 68/200
 8/11 [====================>.........] - ETA: 0s - loss: 4.9863e-04
Epoch 00068: val_loss did not improve from 0.00029
11/11 [==============================] - 0s 10ms/step - loss: 4.6822e-04 - val_loss: 3.2261e-04
Epoch 69/200
 8/11 [====================>.........] - ETA: 0s - loss: 4.2904e-04
Epoch 00069: val_los

Epoch 20/200
157/164 [===========================>..] - ETA: 0s - loss: 4.0737e-04
Epoch 00020: val_loss did not improve from 0.00047
164/164 [==============================] - 1s 5ms/step - loss: 4.0214e-04 - val_loss: 4.8894e-04
Epoch 21/200
157/164 [===========================>..] - ETA: 0s - loss: 4.0588e-04
Epoch 00021: val_loss did not improve from 0.00047
164/164 [==============================] - 1s 5ms/step - loss: 4.0059e-04 - val_loss: 4.6963e-04
Epoch 22/200
157/164 [===========================>..] - ETA: 0s - loss: 4.0531e-04
Epoch 00022: val_loss did not improve from 0.00047
164/164 [==============================] - 1s 5ms/step - loss: 4.0009e-04 - val_loss: 4.6886e-04
Epoch 23/200
157/164 [===========================>..] - ETA: 0s - loss: 4.0316e-04
Epoch 00023: val_loss did not improve from 0.00047
164/164 [==============================] - 1s 5ms/step - loss: 3.9814e-04 - val_loss: 4.7602e-04
Epoch 24/200
157/164 [===========================>..] - ETA: 0s - loss: 4.01

Epoch 53/200
157/164 [===========================>..] - ETA: 0s - loss: 3.8864e-04
Epoch 00053: val_loss did not improve from 0.00045
164/164 [==============================] - 1s 5ms/step - loss: 3.8398e-04 - val_loss: 4.5496e-04
Epoch 54/200
157/164 [===========================>..] - ETA: 0s - loss: 3.8732e-04
Epoch 00054: val_loss did not improve from 0.00045
164/164 [==============================] - 1s 5ms/step - loss: 3.8289e-04 - val_loss: 4.6258e-04
Epoch 55/200
157/164 [===========================>..] - ETA: 0s - loss: 3.8656e-04
Epoch 00055: val_loss did not improve from 0.00045
164/164 [==============================] - 1s 5ms/step - loss: 3.8230e-04 - val_loss: 4.5655e-04
Epoch 56/200
157/164 [===========================>..] - ETA: 0s - loss: 3.8648e-04
Epoch 00056: val_loss did not improve from 0.00045
164/164 [==============================] - 1s 5ms/step - loss: 3.8219e-04 - val_loss: 4.5799e-04
Epoch 57/200
157/164 [===========================>..] - ETA: 0s - loss: 3.87

...

6/6 [==============================] - ETA: 0s - loss: 4.5438e-04
Epoch 00090: val_loss did not improve from 0.00031
6/6 [==============================] - 0s 13ms/step - loss: 4.5438e-04 - val_loss: 4.7340e-04
Epoch 91/200
6/6 [==============================] - ETA: 0s - loss: 5.3029e-04
Epoch 00091: val_loss did not improve from 0.00031
6/6 [==============================] - 0s 13ms/step - loss: 5.3029e-04 - val_loss: 3.6952e-04
Epoch 92/200
6/6 [==============================] - ETA: 0s - loss: 4.4985e-04
Epoch 00092: val_loss improved from 0.00031 to 0.00031, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-003/feature-10/model-092-0.000305.h5
6/6 [==============================] - 0s 16ms/step - loss: 4.4985e-04 - val_loss: 3.0530e-04
Epoch 93/200
1/6 [====>.........................] - ETA: 0s - loss: 2.8910e-04
Epoch 00093: val_loss did not improve from 0.00031
6/6 [==============================] - 0s 13ms/step - loss: 3.9143e-04 - val_loss: 3.5744e-04
Epoch

1/6 [====>.........................] - ETA: 0s - loss: 4.2025e-04
Epoch 00125: val_loss did not improve from 0.00030
6/6 [==============================] - 0s 13ms/step - loss: 4.1309e-04 - val_loss: 3.2546e-04
Epoch 126/200
6/6 [==============================] - ETA: 0s - loss: 3.9704e-04
Epoch 00126: val_loss did not improve from 0.00030
6/6 [==============================] - 0s 13ms/step - loss: 3.9704e-04 - val_loss: 4.1771e-04
Epoch 127/200
6/6 [==============================] - ETA: 0s - loss: 4.5896e-04
Epoch 00127: val_loss did not improve from 0.00030
6/6 [==============================] - 0s 13ms/step - loss: 4.5896e-04 - val_loss: 3.4172e-04
Epoch 128/200
1/6 [====>.........................] - ETA: 0s - loss: 3.1657e-04
Epoch 00128: val_loss did not improve from 0.00030
6/6 [==============================] - 0s 13ms/step - loss: 3.8683e-04 - val_loss: 3.2836e-04
Epoch 129/200
6/6 [==============================] - ETA: 0s - loss: 3.9378e-04
Epoch 00129: val_loss did not impr

Epoch 161/200
6/6 [==============================] - ETA: 0s - loss: 3.9232e-04
Epoch 00161: val_loss did not improve from 0.00029
6/6 [==============================] - 0s 14ms/step - loss: 3.9232e-04 - val_loss: 3.7157e-04
train_score
in test datsaet, zeros: 7154, non_zeros: 8542
in prediction, zeros: 0, non_zeros: 15696
true_positive: 0, true_negative: 8543, false_positive: 0, false_negative: 7154
nrmse: 0.025471, accuracy: 0.544244, f1_score: -1.000000
{'nrmse': 0.02547089530299872, 'accuracy': 0.5442441230808435, 'f1_score': -1}
val_score
in test datsaet, zeros: 2547, non_zeros: 2637
in prediction, zeros: 0, non_zeros: 5184
true_positive: 0, true_negative: 2638, false_positive: 0, false_negative: 2547
nrmse: 0.168773, accuracy: 0.508775, f1_score: -1.000000
{'nrmse': 0.16877311499039258, 'accuracy': 0.5087753134040501, 'f1_score': -1}
test_score
in test datsaet, zeros: 2365, non_zeros: 2819
in prediction, zeros: 0, non_zeros: 5184
true_positive: 0, true_negative: 2820, false_posit

Epoch 23/200
73/82 [=========================>....] - ETA: 0s - loss: 4.0079e-04
Epoch 00023: val_loss improved from 0.00035 to 0.00035, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-003/feature-11/model-023-0.000348.h5
82/82 [==============================] - 0s 5ms/step - loss: 3.9165e-04 - val_loss: 3.4846e-04
Epoch 24/200
73/82 [=========================>....] - ETA: 0s - loss: 3.9764e-04
Epoch 00024: val_loss did not improve from 0.00035
82/82 [==============================] - 0s 5ms/step - loss: 3.8865e-04 - val_loss: 3.5194e-04
Epoch 25/200
73/82 [=========================>....] - ETA: 0s - loss: 3.9865e-04
Epoch 00025: val_loss improved from 0.00035 to 0.00035, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-003/feature-11/model-025-0.000346.h5
82/82 [==============================] - 0s 5ms/step - loss: 3.8937e-04 - val_loss: 3.4567e-04
Epoch 26/200
72/82 [=========================>....] - ETA: 0s - loss: 3.9581e-04
Epoch 00026: v

Epoch 56/200
73/82 [=========================>....] - ETA: 0s - loss: 3.5645e-04
Epoch 00056: val_loss did not improve from 0.00032
82/82 [==============================] - 0s 5ms/step - loss: 3.4737e-04 - val_loss: 3.8780e-04
Epoch 57/200
73/82 [=========================>....] - ETA: 0s - loss: 3.5939e-04
Epoch 00057: val_loss did not improve from 0.00032
82/82 [==============================] - 0s 5ms/step - loss: 3.5030e-04 - val_loss: 3.3554e-04
Epoch 58/200
73/82 [=========================>....] - ETA: 0s - loss: 3.4967e-04
Epoch 00058: val_loss did not improve from 0.00032
82/82 [==============================] - 0s 5ms/step - loss: 3.4156e-04 - val_loss: 3.6969e-04
Epoch 59/200
73/82 [=========================>....] - ETA: 0s - loss: 3.4964e-04
Epoch 00059: val_loss did not improve from 0.00032
82/82 [==============================] - 0s 5ms/step - loss: 3.4027e-04 - val_loss: 3.6951e-04
Epoch 60/200
73/82 [=========================>....] - ETA: 0s - loss: 3.4513e-04
Epoch 00060